# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/falcon9-launches-wiki.png)


## Objectives

*   Extract a Falcon 9 launch records HTML table from Wikipedia
*   Parse the table and convert it into a Pandas data frame


First let's import required packages


In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

Helper Functions

In [2]:
def date_time(table_cells):
    return [date_time.strip() for date_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    out=''.join([booster_version for i,booster_version in enumerate(table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    out=[i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

def extract_column_from_header(row):
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


We will scrape the data from the  List of Falcon 9 and Falcon Heavy launches Wikipedia page

In [3]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

### Requesting the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [4]:
html_data = requests.get(url)
html_data.status_code

200

Creating a BeautifulSoup object from the HTML response


In [5]:
soup = BeautifulSoup(html_data.text, 'html5lib')

Title of the Webpage

In [6]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first.

In [7]:
html_tables = soup.find_all('table')

In [8]:
# First Table
print(html_tables[0].prettify())

<table class="col-begin" role="presentation">
 <tbody>
  <tr>
   <td class="col-break">
    <h3>
     <span class="mw-headline" id="Rocket_configurations">
      Rocket configurations
     </span>
    </h3>
    <div class="chart noresize" style="padding-top:10px;margin-top:1em;max-width:420px;">
     <div style="position:relative;min-height:320px;min-width:420px;max-width:420px;">
      <div style="float:right;position:relative;min-height:240px;min-width:320px;max-width:320px;border-left:1px black solid;border-bottom:1px black solid;">
       <div style="position:absolute;left:3px;top:224px;height:15px;min-width:18px;max-width:18px;background-color:LightSteelBlue;-webkit-print-color-adjust:exact;border:1px solid LightSteelBlue;border-bottom:none;overflow:hidden;" title="[[Falcon 9 v1.0]]: 2">
       </div>
       <div style="position:absolute;left:55px;top:224px;height:15px;min-width:18px;max-width:18px;background-color:LightSteelBlue;-webkit-print-color-adjust:exact;border:1px solid L

In [9]:
# Second Table
print(html_tables[1].prettify())

<table class="col-begin" role="presentation">
 <tbody>
  <tr>
   <td class="col-break">
    <h3>
     <span class="mw-headline" id="Launch_outcomes">
      Launch outcomes
     </span>
    </h3>
    <div class="chart noresize" style="padding-top:10px;margin-top:1em;max-width:480px;">
     <div style="position:relative;min-height:320px;min-width:480px;max-width:480px;">
      <div style="float:right;position:relative;min-height:240px;min-width:380px;max-width:380px;border-left:1px black solid;border-bottom:1px black solid;">
       <div style="position:absolute;left:177px;top:235px;height:4px;min-width:21px;max-width:21px;background-color:Black;-webkit-print-color-adjust:exact;border:1px solid Black;border-bottom:none;overflow:hidden;" title="Loss before launch: 1">
       </div>
       <div style="position:absolute;left:148px;top:235px;height:4px;min-width:21px;max-width:21px;background-color:DarkRed;-webkit-print-color-adjust:exact;border:1px solid DarkRed;border-bottom:none;overflow:

In [10]:
# Third Table
print(html_tables[2].prettify())

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    Flight No.
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     Booster
    </a>
    <sup class="reference" id="cite_ref-booster_11-0">
     <a href="#cite_note-booster-11">
      [b]
     </a>
    </sup>
   </th>
   <th scope="col">
    Launch site
   </th>
   <th scope="col">
    Payload
    <sup class="reference" id="cite_ref-Dragon_12-0">
     <a href="#cite_note-Dragon-12">
      [c]
     </a>
    </sup>
   </th>
   <th scope="col">
    Payload mass
   </th>
   <th scope="col">
    Orbit
   </th>
   <th scope="col">
    Customer
   </th>
   <th scope="col">
    Launch
    <br/>
    outcome
  

In [11]:
first_launch_table = html_tables[2]

Starting from the **third table** is our target table contains the actual launch records.


### Extracting Column Names

Next, we just need to iterate through the <th> elements and apply the provided extract_column_from_header() to extract column name one by one


In [12]:
column_names = []

for element in first_launch_table.find_all('th'):
    name = extract_column_from_header(element)
    print(name)
    if name:
        column_names.append(name)

Flight No.
Date and time ( )

Launch site
Payload
Payload mass
Orbit
Customer
Launch outcome

None
None
None
None
None
None
None


Check the extracted column names


In [13]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [14]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Initialize the launch_dict with an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the launch_dict with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links B0004.1, missing values N/A, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the launch_dict. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [15]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th and rows.th.string:
            flight_number=rows.th.string.strip()
            flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            
            launch_dict['Flight No.'].append(flight_number)
            
            datatimelist=date_time(row[0])
            
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            
            try:
                customer = row[6].a.string
            except:
                customer = 'Various'
            launch_dict['Customer'].append(customer)
            
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            

After you have fill in the parsed launch record values into launch_dict, you can create a dataframe from it.


In [16]:
df=pd.DataFrame(launch_dict)

We can now export it to a **CSV** File.


In [17]:
df.to_csv('spacex_web_scraped.csv', index=False)

In [18]:
# To View it as a DataFrame
df.head(10)

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10
5,6,VAFB,CASSIOPE,500 kg,Polar orbit,MDA,Success,F9 v1.1B1003,Uncontrolled,29 September 2013,16:00
6,7,CCAFS,SES-8,"3,170 kg",GTO,SES,Success,F9 v1.1,No attempt,3 December 2013,22:41
7,8,CCAFS,Thaicom 6,"3,325 kg",GTO,Thaicom,Success,F9 v1.1,No attempt,6 January 2014,22:06
8,9,Cape Canaveral,SpaceX CRS-3,"2,296 kg",LEO,NASA,Success\n,F9 v1.1,Controlled,18 April 2014,19:25
9,10,Cape Canaveral,Orbcomm-OG2,"1,316 kg",LEO,Orbcomm,Success,F9 v1.1,Controlled,14 July 2014,15:15


### Conclusion
We have successfully scraped the Falcon 9 launch records from Wikipedia and stored them in a csv file.